In [1]:
import cv2
import os

def video_to_frames(video, path_output_dir):
    # extract frames from a video and save to directory as 'x.png' where
    # x is the frame index
    vidcap = cv2.VideoCapture(video)
    count = 0
    saved_count = 0  # Variable to keep track of saved frames
    while vidcap.isOpened():
        success, image = vidcap.read()
        if success:
            if count % 3 == 0:  # Save every 3th frame
                cv2.imwrite(os.path.join(path_output_dir, '%d.png') % saved_count, image)
                saved_count += 1
            count += 1
        else:
            break
    cv2.destroyAllWindows()
    vidcap.release()

video_to_frames('dataset/video/2023-11-09 13-16-57.mkv', 'dataset/images')


KeyboardInterrupt: 

In [ ]:
import cv2
import os
import concurrent.futures

def process_video_part(video, path_output_dir, start_frame, end_frame, saved_count):
    vidcap = cv2.VideoCapture(video)
    count = 0
    while count < start_frame:
        success, _ = vidcap.read()
        count += 1
    while count < end_frame and vidcap.isOpened():
        success, image = vidcap.read()
        if success:
            if count % 3 == 0:  # Save every 3rd frame
                cv2.imwrite(os.path.join(path_output_dir, f'{saved_count}.png'), image)
                saved_count += 1
            count += 1
        else:
            break
    vidcap.release()

def video_to_frames_parallel(video, path_output_dir, num_parts=4):
    vidcap = cv2.VideoCapture(video)
    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames_per_part = total_frames // num_parts
    vidcap.release()

    with concurrent.futures.ProcessPoolExecutor() as executor:
        futures = []
        saved_count = 0
        for i in range(num_parts):
            start_frame = i * frames_per_part
            end_frame = (i + 1) * frames_per_part if i < num_parts - 1 else total_frames
            futures.append(executor.submit(process_video_part, video, path_output_dir, start_frame, end_frame, saved_count))
            saved_count += (end_frame - start_frame) // 3

        for future in concurrent.futures.as_completed(futures):
            future.result()

    cv2.destroyAllWindows()

video_to_frames_parallel('dataset/video/2023-11-09 13-16-57.mkv', 'dataset/images', num_parts=4)
